In [37]:
# Настраиваем импорты.
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno

import eda

In [2]:
# Включаем интерактивный режим (для отображения графиков.
%matplotlib inline

In [3]:
# Вводные.
trainPath = 'data/exam_module_4/1_variant_dna_sequence_mutation_prediction/input/train.csv'
testPath = 'data/exam_module_4/1_variant_dna_sequence_mutation_prediction/input/test.csv'
cvFraction = 0.15
randomCeed = 777

In [4]:
# 1. Определяем тип задачи.

In [5]:
print('Task type: logistic regression or SVM')

Task type: logistic regression or SVM


In [6]:
# 2. Создаём фреймы и выделяем часть датасета на CV.

In [7]:
#Смотрим огригинальные тренировочные данные.
originalTrainDf = pd.read_csv(trainPath, index_col='ID')
originalTrainDf.info()
originalTrainDf.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180000 entries, 0 to 299999
Data columns (total 31 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   sequence1  180000 non-null  object 
 1   sequence2  180000 non-null  object 
 2   sequence3  180000 non-null  object 
 3   sequence4  180000 non-null  object 
 4   sequence5  180000 non-null  object 
 5   sequence6  180000 non-null  object 
 6   sequence7  180000 non-null  object 
 7   sequence8  180000 non-null  object 
 8   sequence9  180000 non-null  object 
 9   A          180000 non-null  int64  
 10  B          180000 non-null  int64  
 11  C          180000 non-null  int64  
 12  D          180000 non-null  int64  
 13  E          180000 non-null  int64  
 14  F          180000 non-null  int64  
 15  G          180000 non-null  int64  
 16  H          180000 non-null  int64  
 17  I          180000 non-null  int64  
 18  J          180000 non-null  int64  
 19  K          180000 non-n

,sequence1,sequence2,sequence3,sequence4,sequence5,sequence6,sequence7,sequence8,sequence9,A,...,M,N,O,P,Q,R,S,T,U,mutation
ID,,,,,,,,,,,,,,,,,,,,,
0,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,AAATCAGCTAAA,ATTCCATATTTT,ATTCCATATTTT,AAATCAGCTAAA,8,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
2,ATGCGCTTAATT,ATGCGCTTAATT,GATCGTAGCTAA,ATGCGCTTAATT,ATGCGCTTAATT,AAATCAGCTAAA,ATTCCATATTTT,ATTCCATATTTT,AAATCAGCTAAA,10,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
5,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,AAATCAGCTAAA,ATTCCATATTTT,ATTCCATATTTT,AAATCAGCTAAA,8,...,0.524760,0.580085,0.301498,0.261055,0.193988,0.935688,0.685868,0.277694,0.428115,0
7,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATTCCATATTTT,AAATCAGCTAAA,TAACATACAGAG,TAACATACAGAG,5,...,0.943545,0.209744,0.670577,0.772650,0.845751,0.507320,0.927204,0.605755,0.454155,1
9,ATGCGCTTAATT,GATCGTAGCTAA,ATGCGCTTAATT,GATCGTAGCTAA,GATCGTAGCTAA,ATTCCATATTTT,AAATCAGCTAAA,ATTCCATATTTT,TAACATACAGAG,11,...,0.397622,0.506096,0.200673,0.734145,0.521673,0.365571,0.416094,0.584799,0.447149,1


In [8]:
#Смотрим огригинальные тренировочные данные.
originalTestDf = pd.read_csv(testPath, index_col='ID')
originalTestDf.info()
originalTestDf.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120000 entries, 1 to 299992
Data columns (total 30 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   sequence1  120000 non-null  object 
 1   sequence2  120000 non-null  object 
 2   sequence3  120000 non-null  object 
 3   sequence4  120000 non-null  object 
 4   sequence5  120000 non-null  object 
 5   sequence6  120000 non-null  object 
 6   sequence7  120000 non-null  object 
 7   sequence8  120000 non-null  object 
 8   sequence9  120000 non-null  object 
 9   A          120000 non-null  int64  
 10  B          120000 non-null  int64  
 11  C          120000 non-null  int64  
 12  D          120000 non-null  int64  
 13  E          120000 non-null  int64  
 14  F          120000 non-null  int64  
 15  G          120000 non-null  int64  
 16  H          120000 non-null  int64  
 17  I          120000 non-null  int64  
 18  J          120000 non-null  int64  
 19  K          120000 non-n

,sequence1,sequence2,sequence3,sequence4,sequence5,sequence6,sequence7,sequence8,sequence9,A,...,L,M,N,O,P,Q,R,S,T,U
ID,,,,,,,,,,,,,,,,,,,,,
1,ATGCGCTTAATT,ATGCGCTTAATT,GATCGTAGCTAA,ATGCGCTTAATT,GATCGTAGCTAA,ATTCCATATTTT,AAATCAGCTAAA,ATTCCATATTTT,AAATCAGCTAAA,8,...,0.328929,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729
3,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,AAATCAGCTAAA,ATTCCATATTTT,ATTCCATATTTT,AAATCAGCTAAA,10,...,0.707663,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242
4,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,GATCGTAGCTAA,AAATCAGCTAAA,AAATCAGCTAAA,ATTCCATATTTT,AAATCAGCTAAA,8,...,0.274514,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960
6,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,GATCGTAGCTAA,ATTCCATATTTT,ATTCCATATTTT,ATTCCATATTTT,AAATCAGCTAAA,0,...,0.707840,0.678466,0.747125,0.678212,0.553609,0.281719,0.768782,0.688580,0.320699,0.642256
8,GATCGTAGCTAA,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,AAATCAGCTAAA,ATTCCATATTTT,ATTCCATATTTT,AAATCAGCTAAA,8,...,0.664718,0.635195,0.672264,0.561127,0.419873,0.508550,0.673267,0.356103,0.471519,0.499554


In [9]:
#Формируем датафреймы из псевдорандомных выборок.
trainDf = originalTrainDf.sample(frac=(1 - cvFraction), random_state=randomCeed).drop('mutation', axis=1)
trainDfTarget = originalTrainDf.sample(frac=(1 - cvFraction), random_state=randomCeed)[['mutation']]
cvDf = originalTrainDf.drop(trainDf.index).drop('mutation', axis=1)
cvDfTarget = originalTrainDf.drop(trainDfTarget.index)[['mutation']]
testDf = originalTestDf

In [10]:
print('Train data:')
trainDf.shape

Train data:


(153000, 30)

In [11]:
print('Train target data:')
trainDfTarget.shape

Train target data:


(153000, 1)

In [12]:
print('CV data:')
cvDf.shape

CV data:


(27000, 30)

In [13]:
print('CV target data:')
cvDfTarget.shape

CV target data:


(27000, 1)

In [14]:
print('Test data:')
testDf.shape

Test data:


(120000, 30)

In [15]:
# 3. Определяем тип переменных в датасете.

In [16]:
print('Train data types:')
trainDf.info()
print('Train target data types:')
trainDfTarget.info()

Train data types:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 153000 entries, 176114 to 284756
Data columns (total 30 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   sequence1  153000 non-null  object 
 1   sequence2  153000 non-null  object 
 2   sequence3  153000 non-null  object 
 3   sequence4  153000 non-null  object 
 4   sequence5  153000 non-null  object 
 5   sequence6  153000 non-null  object 
 6   sequence7  153000 non-null  object 
 7   sequence8  153000 non-null  object 
 8   sequence9  153000 non-null  object 
 9   A          153000 non-null  int64  
 10  B          153000 non-null  int64  
 11  C          153000 non-null  int64  
 12  D          153000 non-null  int64  
 13  E          153000 non-null  int64  
 14  F          153000 non-null  int64  
 15  G          153000 non-null  int64  
 16  H          153000 non-null  int64  
 17  I          153000 non-null  int64  
 18  J          153000 non-null  int64  
 19  

In [17]:
# 4. Если это необходимо провести препроцессинг данных, нужно ли применять алгоритмы понижения размерности?
# Нужно ли убирать аномалии?

In [18]:
# Переводим первые 9 столбцов в цифры (по методу one-hot, one-hot столбцы добавляются в конце датафрейма).
dummieCounter = 0
for col in trainDf.columns:
    if trainDf[col].dtypes == object:
        dummieCounter += len(trainDf[col].unique())
        print('Unique in ' + str(col) + ': ' + str(len(trainDf[col].unique())))
print('Dummie columns: ' + str(dummieCounter))

trainDf = pd.get_dummies(trainDf)
cvDf = pd.get_dummies(cvDf)
testDf = pd.get_dummies(testDf)

Unique in sequence1: 2
Unique in sequence2: 2
Unique in sequence3: 2
Unique in sequence4: 2
Unique in sequence5: 2
Unique in sequence6: 4
Unique in sequence7: 4
Unique in sequence8: 4
Unique in sequence9: 4
Dummie columns: 26


In [19]:
print("TrainDF: ")
trainDf.shape

TrainDF: 


(153000, 47)

In [20]:
print('CvDF: ')
cvDf.shape

CvDF: 


(27000, 47)

In [21]:
print("TestDF: ")
testDf.shape

TestDF: 


(120000, 47)

In [22]:
# def time_series_plot(df):
#     """Given dataframe, generate times series plot of numeric data by daily, monthly and yearly frequency"""
#     print("\nTo check time series of numeric data  by daily, monthly and yearly frequency")
#     if len(df.select_dtypes(include='datetime64').columns) > 0:
#         for col in df.select_dtypes(include='datetime64').columns:
#             for p in ['D', 'M', 'Y']:
#                 if p == 'D':
#                     print("Plotting daily data")
#                 elif p == 'M':
#                     print("Plotting monthly data")
#                 else:
#                     print("Plotting yearly data")
#                 for col_num in df.select_dtypes(include=np.number).columns:
#                     __ = df.copy()
#                     __ = __.set_index(col)
#                     __T = __.resample(p).sum()
#                     ax = __T[[col_num]].plot()
#                     ax.set_ylim(bottom=0)
#                     ax.get_yaxis().set_major_formatter(
#                         matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
#                     plt.show()
#
#
# def numeric_eda(df, hue=None):
#     """Given dataframe, generate EDA of numeric data"""
#     print("\nTo check: \nDistribution of numeric data")
#     display(df.describe().T)
#     columns = df.select_dtypes(include=np.number).columns
#     figure = plt.figure(figsize=(20, 10))
#     figure.add_subplot(1, len(columns), 1)
#     for index, col in enumerate(columns):
#         if index > 0:
#             figure.add_subplot(1, len(columns), index + 1)
#         sns.boxplot(y=col, data=df, boxprops={'facecolor': 'None'})
#     figure.tight_layout()
#     plt.show()
#
#     if len(df.select_dtypes(include='category').columns) > 0:
#         for col_num in df.select_dtypes(include=np.number).columns:
#             for col in df.select_dtypes(include='category').columns:
#                 fig = sns.catplot(x=col, y=col_num, kind='violin', data=df, height=5, aspect=2)
#                 fig.set_xticklabels(rotation=90)
#                 plt.show()
#
#     # Plot the pairwise joint distributions
#     print("\nTo check pairwise joint distribution of numeric data")
#     if hue == None:
#         sns.pairplot(df.select_dtypes(include=np.number))
#     else:
#         sns.pairplot(df.select_dtypes(include=np.number).join(df[[hue]]), hue=hue)
#     plt.show()
#
#
# def top5(df):
#     """Given dataframe, generate top 5 unique values for non-numeric data"""
#     columns = df.select_dtypes(include=['object', 'category']).columns
#     for col in columns:
#         print("Top 5 unique values of " + col)
#         print(df[col].value_counts().reset_index().rename(columns={"index": col, col: "Count"})[
#               :min(5, len(df[col].value_counts()))])
#         print(" ")
#
#
# def categorical_eda(df, hue=None):
#     """Given dataframe, generate EDA of categorical data"""
#     print("\nTo check: \nUnique count of non-numeric data\n")
#     print(df.select_dtypes(include=['object', 'category']).nunique())
#     top5(df)
#     # Plot count distribution of categorical data
#     for col in df.select_dtypes(include='category').columns:
#         fig = sns.catplot(x=col, kind="count", data=df, hue=hue)
#         fig.set_xticklabels(rotation=90)
#         plt.show()
#
#
# def eda(df):
#     """Given dataframe, generate exploratory data analysis"""
#     # check that input is pandas dataframe
#     if type(df) != pd.core.frame.DataFrame:
#         raise TypeError("Only pandas dataframe is allowed as input")
#
#     # replace field that's entirely space (or empty) with NaN
#     df = df.replace(r'^\s*$', np.nan, regex=True)
#
#     print("Preview of data:")
#     display(df.head(3))
#
#     print("\nTo check: \n (1) Total number of entries \n (2) Column types \n (3) Any null values\n")
#     print(df.info())
#
#     # generate preview of entries with null values
#     if df.isnull().any(axis=None):
#         print("\nPreview of data with null values:")
#         display(df[df.isnull().any(axis=1)].head(3))
#         missingno.matrix(df)
#         plt.show()
#
#     # generate count statistics of duplicate entries
#     if len(df[df.duplicated()]) > 0:
#         print("\n***Number of duplicated entries: ", len(df[df.duplicated()]))
#         display(df[df.duplicated(keep=False)].sort_values(by=list(df.columns)).head())
#     else:
#         print("\nNo duplicated entries found")
#
#     print('cat')
#     # EDA of categorical data
#     categorical_eda(df)
#
#     print('num')
#     # EDA of numeric data
#     numeric_eda(df)
#     print('time')
#     # Plot time series plot of numeric data
#     time_series_plot(df)

In [51]:
from eda import showinfo
showinfo()

ImportError: cannot import name 'showinfo' from 'eda' (/home/pest/Projects/PythonScripts/eda.py)

In [ ]:
# 5. Провести EDA и вывести какие-то умозаключения и посмотреть на распределения признаков, на корреляции, на выбросы.

In [ ]:
# 6. Подумать над вариантом модели, для того чтобы решить задачу (либо ансамблем моделей)

In [ ]:
print('Так как n << m, лучше использовать логистическую регрессию, либо SMV without kernel.')

In [ ]:
# 7. Подумать нужно ли применять Unsupervised learning подход для решения задачи?
# Неоходима ли дополнительная информация?

In [ ]:
# 8. Обучить модель и вывести валидационный скор по метрике качества.